In [2]:
# Import libraries 

import yaml
import string
import requests

In [3]:
# Recall from the APIs for Data lab that including passwords in code is a terrible practice. 
# So we include a yaml file.

config_file = open('cocktails.yaml', 'r')
config = yaml.safe_load(config_file)

In [8]:
# ascii_lowercase is a pre-initialized string used as string constant
alphabet_string = string.ascii_lowercase

# Generate a list  with the alphabet
alphabet_list = list(alphabet_string)


In [16]:
#  Instantiate a new connection with your config information from above. In this case, API Methods using the developer test key '1' as the API key
cocktails_key = config['cocktails_key']

#  Write our API Request Payloads 
base_url = "https://www.thecocktaildb.com/api/"
json_api = 'json/v1/'
auth = f'{cocktails_key}'



# Start by generating our request bodies. 
# The following code creates a list with the letter of all cocktails, it makes API calls for the compleate endpoint.
request_bodies = []

# Append to the request_bodies list an API call body for each letter in your letters dict. 


for letter in alphabet_list:
    request_bodies.append(base_url + json_api + auth + f"/search.php?f={letter}")

In [18]:
# Loop through each request body
# Make a get request
# Convert the request payload to a dict using the .json() method

data = []

for request in request_bodies:
    data.append(requests.get(request).json())

In [23]:
request_bodies

['https://www.thecocktaildb.com/api/json/v1/1/search.php?f=a',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=b',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=c',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=d',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=e',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=f',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=g',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=h',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=i',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=j',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=k',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=l',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=m',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=n',
 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f=o',
 'https://www.thecocktaildb.com/api/json/v1/1/search.ph

In [20]:
# Because some of the keys in our dict has a "-", we can`t upload the payload to bigquery, so we change the key:
# "strInstructionsZH-HANS", for "strInstructionsZH_HANS" and 
# "strInstructionsZH-HANT" for "strInstructionsZH_HANT"

# Also because there are no cocktails that start with the letter u, we get this message {"drinks":null}
# So we add a if letter['drinks'] != None: to skip this problem.

# Now that we have our payloads, we need to once again parse them into the next jason level. 
drink = []

for letter in data:
    if letter['drinks'] != None: 
        for drinks in letter['drinks']:
            id= drinks['idDrink']
            dato=[id,drinks]
            drink_jaison = dato[1]
            drink_jaison["strInstructionsZH_HANS"] = drink_jaison["strInstructionsZH-HANS"]
            drink_jaison["strInstructionsZH_HANT"] = drink_jaison["strInstructionsZH-HANT"]
            del drink_jaison["strInstructionsZH-HANS"]
            del drink_jaison["strInstructionsZH-HANT"]
            dato[1] = drink_jaison
            drink.append(dato)
            

In [25]:
# Imports the Google Cloud client library
from google.cloud import storage

bucket = config['bucket']

In [26]:
# Instantiates a client
# Now  that we have a service account created and JSON file loaded somewhere, we need to create a client object
client = storage.Client()

In [27]:
# Retrieve an existing bucket
gcs_bucket = client.get_bucket(bucket)

In [29]:
# Define the path within your bucket and the file name
# The blob method create the new file, also an object.
# Now we get a many folders each with an id, names  and ingridients, and store them into a Google cloud bucket in jason format.
import json

for letter in drink:
    path = f"cocktails/{letter[0]}/ingredients.json"
    blob = gcs_bucket.blob(path)
    with blob.open(mode = 'w') as file:
        json.dump(letter[1], file)